In [1]:
import pandas as pd
import numpy as np
import torch
import scripts
from functools import lru_cache
import torchmetrics
from torch import nn
import optuna

/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

First we load the data. The basic idea is to create dictionaries with features associated to the drugs and cell-lines. In principle, the splits and the data shouldn't be changed

In [2]:
@lru_cache(maxsize = None)
def get_data(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset_drugwise(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, test_data))

# Configuration

we declare the configuration, this is going to be model-specific and we get the datasets

In [3]:
config = {"features" : {"fp_radius":2},
           "optimizer": {"batch_size": 4,
                         "clip_norm":19,
                         "learning_rate":0.00004592646200179472,
                         "stopping_patience":15,
                         "c_alpha":0.01},
           "model":{"embed_dim":485,
                  "hidden_dim":696,
                  "dropout":0.48541242824674574,
                  "n_layers": 4,
                  "norm": "batchnorm"},
          "env": {"fold": 0,
                  "device":"cpu",
                  "max_epochs": 100,
                  "search_hyperparameters":True}}

In [4]:
train_dataset, validation_dataset, test_dataset = get_data(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])



[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerator
[13:34:29] DEPRECATION WARNING: please use MorganGenerat

In [5]:
cell_features, drug_features, target, drug_index, cell_indices, labels = train_dataset[4]

ins = train_dataset[7]
print(ins[5].shape)
print(ins[4].shape)

print("Drug Features Shape:", drug_features.shape)
print("Cell Features Shape:", cell_features.shape)  
print("Labels Shape:", labels.shape)
print("Target Shape:", target.shape)               
print("Drug Index Shape:", drug_index.shape)      
print("Cell Indices Shape:", cell_indices.shape) 




torch.Size([325])
torch.Size([325])
Drug Features Shape: torch.Size([323, 2048])
Cell Features Shape: torch.Size([323, 777])
Labels Shape: torch.Size([323])
Target Shape: torch.Size([323])
Drug Index Shape: torch.Size([323])
Cell Indices Shape: torch.Size([323])


In [6]:

def collate_fn_custom(batch):
    omics = []
    drugs = []
    targets = []
    cell_ids = []
    drug_ids = []
    labels = [] 

    for i, (o, d, t, cid, did, r) in enumerate(batch):
        omics.append(o)  # Add each omics tensor
        drugs.append(d)  # Add each drugs tensor
        targets.append(t)  # Add each target tensor
        cell_ids.append(cid)  # Add each cell ID tensor
        drug_ids.append(did)  # Add each drug ID tensor
        r = r.clone()
        r[r==1] = 2*i + 1
        r[r==2] = 2*(i + 1)
        labels.append(r)  # Add each label tensor
    # Concatenate along the first dimension for all tensors
    omics = torch.cat(omics, dim=0)
    drugs = torch.cat(drugs, dim=0)
    targets = torch.cat(targets, dim=0)
    cell_ids = torch.cat(cell_ids, dim=0)
    drug_ids = torch.cat(drug_ids, dim=0)
    labels = torch.cat(labels, dim=0)
    


    return omics, drugs, targets, cell_ids, drug_ids, labels


# Hyperparameter optimization

we wrap the function for training the model in a function that can be used by optuna

In [ ]:
def train_model_optuna(trial, config):
    def pruning_callback(epoch, train_r):
        trial.report(train_r, step = epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()
    config["model"] = {"embed_dim": trial.suggest_int("embed_dim", 64, 512),
                    "hidden_dim": trial.suggest_int("hidden_dim", 64, 2048),
                    "n_layers": trial.suggest_int("n_layers", 1, 6),
                    "norm": trial.suggest_categorical("norm", ["batchnorm", "layernorm", None]),
                    "dropout": trial.suggest_float("dropout", 0.0, 0.5)}
    config["optimizer"] = { "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
                            "clip_norm": trial.suggest_int("clip_norm", 0.1, 20),
                            "batch_size": trial.suggest_int("batch_size", 6, 12),
                            "stopping_patience":10,
                            "c_alpha": trial.suggest_float("c_alpha", 0.001, 0.01)}
    try:
        R, model = scripts.train_model(config,
                                       train_dataset,
                                       validation_dataset,
                                       use_momentum=True,
                                       callback_epoch = pruning_callback,
                                       collate_fn=collate_fn_custom)
        
        return R
    except Exception as e:
        print(e)
        return 0

In [8]:
import scripts

%reload_ext autoreload

%autoreload 2



if config["env"]["search_hyperparameters"]:
    study_name = f"baseline_model"
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    study = optuna.create_study(study_name=study_name,
                                storage=storage_name,
                                direction='maximize',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=30,
                                                               n_warmup_steps=5,
                                                               interval_steps=5))
    objective = lambda x: train_model_optuna(x, config)
    study.optimize(objective, n_trials=40)
    best_config = study.best_params
    print(best_config)
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["norm"] = best_config["norm"]
    config["model"]["dropout"] = best_config["dropout"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]
    config["model"]["c_alpha"] = best_config["c_alpha"]



[I 2025-02-18 13:35:58,374] Using an existing study with name 'baseline_model' instead of creating a new one.
/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028
/home/karolyi/PROJECT_repo/scripts/models.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered

[I 2025-02-18 13:38:07,788] Trial 777 finished with value: 0.0 and parameters: {'embed_dim': 443, 'hidden_dim': 408, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.09336403656761488, 'learning_rate': 7.22585627340308e-05, 'clip_norm': 18, 'batch_size': 3, 'c_alpha': 0.005228611982399028}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 13:38:43,244] Trial 778 finished with value: 0.0 and parameters: {'embed_dim': 139, 'hidden_dim': 277, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.08032180873572226, 'learning_rate': 0.00010854875257767366, 'clip_norm': 16, 'batch_size': 5, 'c_alpha': 0.01381401252088977}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 13:41:52,282] Trial 779 finished with value: 0.0 and parameters: {'embed_dim': 130, 'hidden_dim': 459, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.12806210157605477, 'learning_rate': 0.00017079677485126306, 'clip_norm': 18, 'batch_size': 4, 'c_alpha': 0.004380824517558049}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 13:42:33,831] Trial 780 finished with value: 0.0 and parameters: {'embed_dim': 248, 'hidden_dim': 228, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.1035554794599842, 'learning_rate': 8.599733082119479e-05, 'clip_norm': 17, 'batch_size': 3, 'c_alpha': 0.017365451440899835}. Best is trial 752 with value: 0.34190896010425487.


...

[I 2025-02-18 14:29:38,000] Trial 812 finished with value: 0.0 and parameters: {'embed_dim': 230, 'hidden_dim': 90, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.08966500936246546, 'learning_rate': 0.00018270251827991145, 'clip_norm': 5, 'batch_size': 4, 'c_alpha': 0.040399160905909316}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 14:30:15,792] Trial 813 finished with value: 0.0 and parameters: {'embed_dim': 158, 'hidden_dim': 394, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.10099837726346586, 'learning_rate': 0.00012181454970579427, 'clip_norm': 0, 'batch_size': 6, 'c_alpha': 0.015664102161016262}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 14:31:04,758] Trial 814 finished with value: 0.0 and parameters: {'embed_dim': 273, 'hidden_dim': 71, 'n_layers': 6, 'norm': 'batchnorm', 'dropout': 0.4007792298452206, 'learning_rate': 6.926601450217504e-05, 'clip_norm': 4, 'batch_size': 6, 'c_alpha': 0.013347615214050139}. Best is trial 752 with value: 0.34190896010425487.


[I 2025-02-18 14:31:41,045] Trial 815 finished with value: 0.0 and parameters: {'embed_dim': 115, 'hidden_dim': 242, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.37339359270857464, 'learning_rate': 0.00011183966087362417, 'clip_norm': 2, 'batch_size': 6, 'c_alpha': 0.0019682467751211204}. Best is trial 752 with value: 0.34190896010425487.



{'embed_dim': 461, 'hidden_dim': 331, 'n_layers': 1, 'norm': 'batchnorm', 'dropout': 0.08924372196854445, 'learning_rate': 0.00012309642970619275, 'clip_norm': 17, 'batch_size': 3, 'c_alpha': 0.0010341689454045453}


# Model training and evaluation

After we have a set of optimal hyperparameters we train our model. The train model function could be changed, but:
- test_dataset cannot be used until we call the final evaluation step
- the evaluation step cannot be modified, it must take the model produced by your pipeline, a dataloader that provides the correct data for your model, and the final metrics have to be printed

In [10]:
import scripts

%reload_ext autoreload

%autoreload 2

_, model = scripts.train_model(config, train_dataset,  validation_dataset=None, use_momentum=True, callback_epoch=None, collate_fn=collate_fn_custom)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                      batch_size=config["optimizer"]["batch_size"],
                                      drop_last=False,
                                      shuffle=False,
                                      collate_fn=collate_fn_custom)


epoch : 0: train loss: 7.993910234708053 Smoothed R interaction (validation) None
epoch : 1: train loss: 6.8697050248201075 Smoothed R interaction (validation) None
epoch : 2: train loss: 6.528052566143183 Smoothed R interaction (validation) None
epoch : 3: train loss: 6.073817754021058 Smoothed R interaction (validation) None
epoch : 4: train loss: 5.35983569175005 Smoothed R interaction (validation) None
epoch : 5: train loss: 4.56787799184139 Smoothed R interaction (validation) None
epoch : 6: train loss: 4.327051956493121 Smoothed R interaction (validation) None
epoch : 7: train loss: 3.9188241351109285 Smoothed R interaction (validation) None
epoch : 8: train loss: 3.9119853199674535 Smoothed R interaction (validation) None
epoch : 9: train loss: 3.48958902576795 Smoothed R interaction (validation) None


KeyboardInterrupt: 

In [16]:
final_metrics = scripts.evaluate_step(model, test_dataloader, metrics, device)
print(final_metrics)

/home/karolyi/PROJECT_repo/scripts/models.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1730580467430/work/aten/src/ATen/native/TensorShape.cpp:3683.)
  return torch.linalg.solve(A, Xy).T


{'MSE': 2.363308906555176, 'R_cellwise': 0.2313278466463089, 'R_cellwise_residuals': 0.21419179439544678}
